In [ ]:
# Q1-3, analysis_final
"""
Longitudinal Category Selectivity Analysis
OTC vs nonOTC vs Control - Bilateral vs Unilateral Categories
"""

import pandas as pd
import numpy as np

# ============================================================
# CONFIGURATION
# ============================================================
FILE_PATH = '/user_data/csimmon2/git_repos/long_pt/results_final_corrected.csv'

MEASURES = {
    'Selectivity_Change': 'Selectivity',
    'Geometry_Preservation_6mm': 'Geometry',
    'Spatial_Relocation_mm': 'Spatial Drift',
    'MDS_Shift': 'MDS'
}

CATEGORIES = ['Face', 'Word', 'Object', 'House']

# ============================================================
# DATA LOADING & PROCESSING
# ============================================================
def load_and_process(filepath):
    """Load data and average controls across hemispheres."""
    df = pd.read_csv(filepath)
    
    controls = df[df['Group'] == 'control'].copy()
    patients = df[df['Group'] != 'control'].copy()
    
    numeric_cols = list(MEASURES.keys()) + ['age_1', 'yr_gap']
    
    # Average controls across L/R hemispheres
    controls_agg = controls.groupby(['Subject', 'Category'])[numeric_cols].mean().reset_index()
    meta = controls.groupby(['Subject', 'Category'])[['Group', 'Category_Type']].first().reset_index()
    controls_final = pd.merge(controls_agg, meta, on=['Subject', 'Category'])
    
    common_cols = ['Subject', 'Group', 'Category', 'Category_Type'] + numeric_cols
    return pd.concat([patients[common_cols], controls_final[common_cols]], ignore_index=True)


def compute_instability_gap(df, measure):
    """Compute Bilateral - Unilateral gap per subject."""
    df_filtered = df[df['Category'].isin(CATEGORIES)].copy()
    subj_means = df_filtered.groupby(['Subject', 'Group', 'Category_Type'])[measure].mean().reset_index()
    pivot = subj_means.pivot(index=['Subject', 'Group'], columns='Category_Type', values=measure).reset_index()
    
    if 'Bilateral' in pivot.columns and 'Unilateral' in pivot.columns:
        pivot['Gap'] = pivot['Bilateral'] - pivot['Unilateral']
        return pivot.dropna(subset=['Gap'])
    return pd.DataFrame()


# ============================================================
# BOOTSTRAP STATISTICS
# ============================================================
def bootstrap_compare(group_a, group_b, n_boot=100000, seed=42):
    """
    Non-parametric bootstrap comparison.
    Returns: dict with diff, ci_low, ci_high, p_value, is_sig
    """
    np.random.seed(seed)
    n_a, n_b = len(group_a), len(group_b)
    
    obs_diff = np.mean(group_a) - np.mean(group_b)
    
    sample_a = np.random.choice(group_a, size=(n_boot, n_a), replace=True)
    sample_b = np.random.choice(group_b, size=(n_boot, n_b), replace=True)
    boot_diffs = np.mean(sample_a, axis=1) - np.mean(sample_b, axis=1)
    
    ci_low, ci_high = np.percentile(boot_diffs, [2.5, 97.5])
    
    # Two-tailed p-value: proportion of bootstrap samples on wrong side of zero
    p_value = 2 * min(np.mean(boot_diffs > 0), np.mean(boot_diffs < 0))
    p_value = min(p_value, 1.0)  # Cap at 1.0
    
    is_sig = (ci_low > 0 and ci_high > 0) or (ci_low < 0 and ci_high < 0)
    
    return {
        'mean_a': np.mean(group_a),
        'mean_b': np.mean(group_b),
        'diff': obs_diff,
        'ci_low': ci_low,
        'ci_high': ci_high,
        'p_value': p_value,
        'is_significant': is_sig,
        'n_a': n_a,
        'n_b': n_b
    }


# ============================================================
# ANALYSIS FUNCTIONS
# ============================================================
def run_q1_instability_gap(df):
    """Q1: Test if OTC shows larger bilateral-unilateral gap than controls/nonOTC."""
    results = []
    
    for col, name in MEASURES.items():
        gaps = compute_instability_gap(df, col)
        
        otc = gaps[gaps['Group'] == 'OTC']['Gap'].values
        ctrl = gaps[gaps['Group'] == 'control']['Gap'].values
        nonotc = gaps[gaps['Group'] == 'nonOTC']['Gap'].values
        
        for comp_name, comp_data in [('vs Control', ctrl), ('vs nonOTC', nonotc)]:
            if len(otc) == 0 or len(comp_data) == 0:
                continue
            
            stats = bootstrap_compare(otc, comp_data)
            results.append({
                'Measure': name,
                'Comparison': comp_name,
                'OTC_Gap': stats['mean_a'],
                'Ref_Gap': stats['mean_b'],
                'Difference': stats['diff'],
                'CI_Low': stats['ci_low'],
                'CI_High': stats['ci_high'],
                'p_value': stats['p_value'],
                'Significant': '*' if stats['is_significant'] else ''
            })
    
    return pd.DataFrame(results)


def run_q2_category_specificity(df):
    """Q2: Test category-specific effects (OTC vs nonOTC)."""
    results = []
    
    for col, name in MEASURES.items():
        for cat in CATEGORIES:
            df_cat = df[df['Category'] == cat]
            
            otc = df_cat[df_cat['Group'] == 'OTC'][col].values
            nonotc = df_cat[df_cat['Group'] == 'nonOTC'][col].values
            
            if len(otc) == 0 or len(nonotc) == 0:
                continue
            
            stats = bootstrap_compare(otc, nonotc)
            cat_type = 'Bilateral' if cat in ['Object', 'House'] else 'Unilateral'
            
            results.append({
                'Measure': name,
                'Category': cat,
                'Category_Type': cat_type,
                'OTC': stats['mean_a'],
                'nonOTC': stats['mean_b'],
                'Difference': stats['diff'],
                'CI_Low': stats['ci_low'],
                'CI_High': stats['ci_high'],
                'p_value': stats['p_value'],
                'Significant': '*' if stats['is_significant'] else ''
            })
    
    return pd.DataFrame(results)


def run_q3_within_group_laterality(df):
    """Q3: Test bilateral vs unilateral WITHIN each group."""
    results = []
    
    for col, name in MEASURES.items():
        for group in ['OTC', 'nonOTC', 'control']:
            group_df = df[df['Group'] == group]
            
            bilateral = group_df[group_df['Category_Type'] == 'Bilateral'][col].values
            unilateral = group_df[group_df['Category_Type'] == 'Unilateral'][col].values
            
            if len(bilateral) == 0 or len(unilateral) == 0:
                continue
            
            stats = bootstrap_compare(bilateral, unilateral)
            
            results.append({
                'Measure': name,
                'Group': group,
                'Bilateral': stats['mean_a'],
                'Unilateral': stats['mean_b'],
                'Difference': stats['diff'],
                'CI_Low': stats['ci_low'],
                'CI_High': stats['ci_high'],
                'p_value': stats['p_value'],
                'Significant': '*' if stats['is_significant'] else ''
            })
    
    return pd.DataFrame(results)


def format_p(p):
    """Format p-value for display."""
    if p < .001:
        return '<.001'
    elif p < .01:
        return f'{p:.3f}'
    else:
        return f'{p:.3f}'


def print_results(q1_df, q2_df, q3_df):
    """Pretty print results."""
    print("=" * 105)
    print("Q1: INSTABILITY GAP  [Mean(House,Object) - Mean(Face,Word)]")
    print("    Tests if OTC's gap is LARGER than controls/nonOTC")
    print("=" * 105)
    print(f"{'Measure':<14} {'Comparison':<12} {'OTC':>7} {'Ref':>7} {'Diff':>7}  {'95% CI':<18} {'p':>8}  Sig")
    print("-" * 105)
    
    for _, row in q1_df.iterrows():
        print(f"{row['Measure']:<14} {row['Comparison']:<12} {row['OTC_Gap']:>7.3f} {row['Ref_Gap']:>7.3f} "
              f"{row['Difference']:>7.3f}  [{row['CI_Low']:>6.3f}, {row['CI_High']:>6.3f}] {format_p(row['p_value']):>8}  {row['Significant']}")
    
    print("\n" + "=" * 105)
    print("Q2: CATEGORY SPECIFICITY (OTC vs nonOTC)")
    print("=" * 105)
    
    for measure in MEASURES.values():
        print(f"\n--- {measure} ---")
        measure_df = q2_df[q2_df['Measure'] == measure]
        for _, row in measure_df.iterrows():
            print(f"  {row['Category']:<8} ({row['Category_Type'][:3]})  "
                  f"OTC={row['OTC']:.3f}  nonOTC={row['nonOTC']:.3f}  "
                  f"diff={row['Difference']:>6.3f}  [{row['CI_Low']:>6.3f},{row['CI_High']:>6.3f}]  "
                  f"p={format_p(row['p_value'])}  {row['Significant']}")
    
    print("\n" + "=" * 105)
    print("Q3: BILATERAL vs UNILATERAL (within each group)")
    print("    Tests if bilateral categories are MORE unstable than unilateral WITHIN each group")
    print("=" * 105)
    print(f"{'Measure':<14} {'Group':<10} {'Bilat':>7} {'Unilat':>7} {'Diff':>7}  {'95% CI':<18} {'p':>8}  Sig")
    print("-" * 105)
    
    for _, row in q3_df.iterrows():
        print(f"{row['Measure']:<14} {row['Group']:<10} {row['Bilateral']:>7.3f} {row['Unilateral']:>7.3f} "
              f"{row['Difference']:>7.3f}  [{row['CI_Low']:>6.3f}, {row['CI_High']:>6.3f}] {format_p(row['p_value']):>8}  {row['Significant']}")


# ============================================================
# MAIN EXECUTION
# ============================================================
if __name__ == "__main__":
    # Load and process data
    df = load_and_process(FILE_PATH)
    print(f"Loaded {len(df)} rows\n")
    
    # Run analyses
    q1_results = run_q1_instability_gap(df)
    q2_results = run_q2_category_specificity(df)
    q3_results = run_q3_within_group_laterality(df)
    
    # Print results
    print_results(q1_results, q2_results, q3_results)
    
    # Summary
    print("\n" + "=" * 95)
    print("SUMMARY")
    print("=" * 95)
    print("""
Q1: Tests if OTC's bilateral-unilateral GAP is larger than other groups
Q2: Tests which specific categories drive the OTC effect  
Q3: Tests if bilateral > unilateral WITHIN each group (the core hypothesis)

* = 95% CI excludes zero (significant effect)
""")

Loaded 96 rows

Q1: INSTABILITY GAP  [Mean(House,Object) - Mean(Face,Word)]
    Tests if OTC's gap is LARGER than controls/nonOTC
Measure        Comparison       OTC     Ref    Diff  95% CI                    p  Sig
---------------------------------------------------------------------------------------------------------
Selectivity    vs Control     0.275   0.109   0.166  [ 0.019,  0.337]    0.024  *
Selectivity    vs nonOTC      0.275   0.008   0.267  [ 0.132,  0.428]    <.001  *
Geometry       vs Control    -0.289  -0.096  -0.193  [-0.340, -0.043]    0.012  *
Geometry       vs nonOTC     -0.289  -0.036  -0.253  [-0.404, -0.103]    <.001  *
Spatial Drift  vs Control    -5.535  -0.344  -5.191  [-13.376,  2.799]    0.214  
Spatial Drift  vs nonOTC     -5.535  -1.826  -3.709  [-11.513,  3.960]    0.359  
MDS            vs Control     0.077   0.020   0.056  [ 0.002,  0.116]    0.040  *
MDS            vs nonOTC      0.077   0.044   0.033  [-0.025,  0.095]    0.287  

Q2: CATEGORY SPECIFICI

In [ ]:
# Q4: PAIRWISE CATEGORY COMPARISONS
"""
Run this cell AFTER analysis_final
Tests if bilateral vs unilateral effect is driven by specific categories
"""

def format_p(p):
    """Format p-value for display."""
    if p < .001:
        return '<.001'
    else:
        return f'{p:.3f}'

# ============================================================
# Q4: PAIRWISE CATEGORY COMPARISONS
# ============================================================
def run_q4_pairwise_categories(df, group='OTC'):
    """Pairwise: each bilateral category vs each unilateral category."""
    results = []
    bilateral_cats = ['Object', 'House']
    unilateral_cats = ['Face', 'Word']
    group_df = df[df['Group'] == group]
    
    for col, name in MEASURES.items():
        for bil_cat in bilateral_cats:
            for uni_cat in unilateral_cats:
                bil_vals = group_df[group_df['Category'] == bil_cat][col].values
                uni_vals = group_df[group_df['Category'] == uni_cat][col].values
                if len(bil_vals) == 0 or len(uni_vals) == 0:
                    continue
                stats = bootstrap_compare(bil_vals, uni_vals)
                results.append({
                    'Measure': name,
                    'Comparison': f"{bil_cat} vs {uni_cat}",
                    'Bilateral_Cat': bil_cat,
                    'Unilateral_Cat': uni_cat,
                    'Bilateral': stats['mean_a'],
                    'Unilateral': stats['mean_b'],
                    'Difference': stats['diff'],
                    'CI_Low': stats['ci_low'],
                    'CI_High': stats['ci_high'],
                    'p_value': stats['p_value'],
                    'Significant': '*' if stats['is_significant'] else ''
                })
    return pd.DataFrame(results)

# Run
q4_results = run_q4_pairwise_categories(df, group='OTC')

# Print
print("=" * 110)
print("Q4: PAIRWISE CATEGORY COMPARISONS (within OTC)")
print("    Tests which bilateral-unilateral pairs drive the effect")
print("=" * 110)

for measure in MEASURES.values():
    print(f"\n--- {measure} ---")
    measure_df = q4_results[q4_results['Measure'] == measure]
    for _, row in measure_df.iterrows():
        print(f"  {row['Comparison']:<18}  {row['Bilateral']:.3f} vs {row['Unilateral']:.3f}  "
              f"diff={row['Difference']:>7.3f}  [{row['CI_Low']:>6.3f},{row['CI_High']:>6.3f}]  "
              f"p={format_p(row['p_value'])}  {row['Significant']}")

# Summary
print("\n" + "=" * 110)
print("SUMMARY: Significant comparisons per category")
print("=" * 110)
for cat in ['Object', 'House']:
    n = (q4_results[q4_results['Bilateral_Cat'] == cat]['Significant'] == '*').sum()
    print(f"  {cat:<8} (Bil): {n}/8 significant")
for cat in ['Face', 'Word']:
    n = (q4_results[q4_results['Unilateral_Cat'] == cat]['Significant'] == '*').sum()
    print(f"  {cat:<8} (Uni): {n}/8 significant")

Q4: PAIRWISE CATEGORY COMPARISONS (within OTC)
    Tests which bilateral-unilateral pairs drive the effect

--- Selectivity ---
  Object vs Face      0.490 vs 0.167  diff=  0.323  [ 0.067, 0.576]  p=0.012  *
  Object vs Word      0.490 vs 0.120  diff=  0.369  [ 0.115, 0.619]  p=0.003  *
  House vs Face       0.348 vs 0.167  diff=  0.181  [ 0.009, 0.333]  p=0.040  *
  House vs Word       0.348 vs 0.120  diff=  0.228  [ 0.060, 0.375]  p=0.009  *

--- Geometry ---
  Object vs Face      0.636 vs 0.751  diff= -0.115  [-0.332, 0.119]  p=0.324  
  Object vs Word      0.636 vs 0.674  diff= -0.039  [-0.246, 0.182]  p=0.715  
  House vs Face       0.212 vs 0.751  diff= -0.539  [-0.806,-0.235]  p=<.001  *
  House vs Word       0.212 vs 0.674  diff= -0.462  [-0.722,-0.166]  p=0.003  *

--- Spatial Drift ---
  Object vs Face      5.869 vs 6.543  diff= -0.673  [-5.198, 3.997]  p=0.771  
  Object vs Word      5.869 vs 18.430  diff=-12.560  [-24.284,-1.745]  p=0.019  *
  House vs Face       8.033 vs 6

In [13]:
# Q5: age and gap interactions
"""
Q5: COVARIATE ANALYSIS
Run this cell AFTER analysis_final.py
Tests if age or scan gap correlate with bilateral-unilateral difference
"""

from scipy import stats

# ============================================================
# Q5: COVARIATE ANALYSIS
# ============================================================
def compute_subject_gap(df, measure):
    """Compute Bilateral - Unilateral gap per subject, with covariates."""
    subj = df.groupby(['Subject', 'Group', 'Category_Type']).agg({
        measure: 'mean',
        'age_1': 'first',
        'yr_gap': 'first'
    }).reset_index()
    
    pivot = subj.pivot(index=['Subject', 'Group', 'age_1', 'yr_gap'], 
                       columns='Category_Type', values=measure).reset_index()
    
    if 'Bilateral' in pivot.columns and 'Unilateral' in pivot.columns:
        pivot['Gap'] = pivot['Bilateral'] - pivot['Unilateral']
        return pivot.dropna(subset=['Gap'])
    return pd.DataFrame()


def run_q5_covariates(df):
    """Correlate age and scan gap with bilateral-unilateral gap per group."""
    results = []
    
    for col, name in MEASURES.items():
        gaps = compute_subject_gap(df, col)
        
        for group in ['OTC', 'nonOTC', 'control']:
            group_gaps = gaps[gaps['Group'] == group]
            
            if len(group_gaps) < 4:  # Need minimum for correlation
                continue
            
            # Age correlation
            r_age, p_age = stats.pearsonr(group_gaps['age_1'], group_gaps['Gap'])
            
            # Scan gap correlation
            r_gap, p_gap = stats.pearsonr(group_gaps['yr_gap'], group_gaps['Gap'])
            
            results.append({
                'Measure': name,
                'Group': group,
                'n': len(group_gaps),
                'r_age': r_age,
                'p_age': p_age,
                'r_scangap': r_gap,
                'p_scangap': p_gap
            })
    
    return pd.DataFrame(results)


# Run
q5_results = run_q5_covariates(df)

# Print
print("=" * 100)
print("Q5: COVARIATE ANALYSIS")
print("    Correlations between covariates and bilateral-unilateral gap")
print("=" * 100)
print(f"{'Measure':<14} {'Group':<10} {'n':>3}  {'r_age':>7} {'p_age':>8}  {'r_gap':>7} {'p_gap':>8}")
print("-" * 100)

for _, row in q5_results.iterrows():
    age_sig = '*' if row['p_age'] < .05 else ''
    gap_sig = '*' if row['p_scangap'] < .05 else ''
    print(f"{row['Measure']:<14} {row['Group']:<10} {row['n']:>3}  "
          f"{row['r_age']:>7.3f} {row['p_age']:>7.3f}{age_sig}  "
          f"{row['r_scangap']:>7.3f} {row['p_scangap']:>7.3f}{gap_sig}")

# Summary
print("\n" + "=" * 100)
print("INTERPRETATION")
print("=" * 100)
print("""
r_age: Correlation between age at T1 and (Bilateral - Unilateral) gap
r_gap: Correlation between interscan interval and (Bilateral - Unilateral) gap

Significant correlation would suggest the effect is modulated by age or time between scans.
No significant correlations = effect is consistent across ages/intervals.
""")

Q5: COVARIATE ANALYSIS
    Correlations between covariates and bilateral-unilateral gap
Measure        Group        n    r_age    p_age    r_gap    p_gap
----------------------------------------------------------------------------------------------------
Selectivity    OTC          6    0.700   0.121   -0.022   0.967
Selectivity    nonOTC       9   -0.254   0.510    0.449   0.225
Selectivity    control      9   -0.222   0.566    0.142   0.716
Geometry       OTC          6    0.248   0.636    0.413   0.416
Geometry       nonOTC       9    0.856   0.003*    0.347   0.361
Geometry       control      9   -0.307   0.421    0.261   0.497
Spatial Drift  OTC          6    0.253   0.629    0.125   0.814
Spatial Drift  nonOTC       9    0.393   0.296    0.302   0.430
Spatial Drift  control      9    0.841   0.004*   -0.169   0.664
MDS            OTC          6   -0.837   0.038*   -0.500   0.313
MDS            nonOTC       9    0.391   0.298    0.141   0.717
MDS            control      9    0.202

In [14]:
# Q6: Check Multicollinearity

"""
Q6: MEASURE INDEPENDENCE (Multicollinearity Check)
Run this cell AFTER analysis_final.py
Tests if Selectivity, Geometry, and MDS are independent measures
"""

from scipy import stats

# ============================================================
# Q6: MEASURE INDEPENDENCE
# ============================================================
def run_q6_multicollinearity(df, group='OTC'):
    """Check correlations between measures within a group."""
    group_df = df[df['Group'] == group].dropna(subset=[
        'Selectivity_Change', 'Geometry_Preservation_6mm', 'MDS_Shift'
    ])
    
    measures = {
        'Selectivity': 'Selectivity_Change',
        'Geometry': 'Geometry_Preservation_6mm', 
        'MDS': 'MDS_Shift'
    }
    
    results = []
    pairs = [('Selectivity', 'Geometry'), ('Selectivity', 'MDS'), ('Geometry', 'MDS')]
    
    for m1, m2 in pairs:
        r, p = stats.pearsonr(group_df[measures[m1]], group_df[measures[m2]])
        results.append({
            'Pair': f'{m1} vs {m2}',
            'r': r,
            'p': p,
            'r_squared': r**2,
            'Concern': '⚠️' if abs(r) > 0.7 else '✓'
        })
    
    return pd.DataFrame(results), len(group_df)


# Run for each group
print("=" * 80)
print("Q6: MEASURE INDEPENDENCE (Multicollinearity)")
print("    |r| > 0.7 suggests measures may be redundant")
print("=" * 80)

for group in ['OTC', 'nonOTC', 'control']:
    results, n = run_q6_multicollinearity(df, group)
    print(f"\n--- {group} (n={n} observations) ---")
    print(f"{'Pair':<22} {'r':>8} {'p':>8} {'r²':>8} {'Status':>8}")
    print("-" * 60)
    for _, row in results.iterrows():
        print(f"{row['Pair']:<22} {row['r']:>8.3f} {row['p']:>8.3f} {row['r_squared']:>8.3f} {row['Concern']:>8}")

print("\n" + "=" * 80)
print("INTERPRETATION")
print("=" * 80)
print("""
r² < 0.50 (|r| < 0.7): Measures capture independent variance — GOOD
r² > 0.50 (|r| > 0.7): Measures may be redundant — CONCERN

Low correlations = converging evidence from independent measures.
""")


Q6: MEASURE INDEPENDENCE (Multicollinearity)
    |r| > 0.7 suggests measures may be redundant

--- OTC (n=24 observations) ---
Pair                          r        p       r²   Status
------------------------------------------------------------
Selectivity vs Geometry   -0.057    0.790    0.003        ✓
Selectivity vs MDS        0.402    0.052    0.161        ✓
Geometry vs MDS          -0.248    0.242    0.062        ✓

--- nonOTC (n=36 observations) ---
Pair                          r        p       r²   Status
------------------------------------------------------------
Selectivity vs Geometry   -0.033    0.850    0.001        ✓
Selectivity vs MDS       -0.123    0.477    0.015        ✓
Geometry vs MDS          -0.377    0.023    0.142        ✓

--- control (n=36 observations) ---
Pair                          r        p       r²   Status
------------------------------------------------------------
Selectivity vs Geometry   -0.161    0.349    0.026        ✓
Selectivity vs MDS      

# FIGURE PLAN

# Figure Plan: Longitudinal Bilateral Category Instability

## Figure 1: Lesion Anatomy & Patient Classification

**Purpose**: Establish OTC vs nonOTC distinction; show lesion heterogeneity

**Layout**: 
- Panel A: 6 OTC patients — axial slices showing posterior VTC involvement
- Panel B: Representative nonOTC patients — anterior temporal resections sparing VTC
- Panel C: Schematic showing VTC ROIs (FFA, VWFA, LOC, PPA) with resection overlay

**Look-alike**: Liu et al. 2025, Fig 1 — structural MRIs with colored ROI overlays per patient
- Shows postoperative scans with timeline
- Color-coded by patient (shapes: triangle, circle, square, diamond, star)

**Key elements**:
- Native space (no normalization)
- Consistent slice selection across patients
- Lesion boundary clearly visible
- Label each patient (OTC004, OTC008, etc.)

---

## Figure 2: Main Result — Bilateral vs Unilateral by Group

**Purpose**: Core finding — bilateral categories degrade in OTC only

**Layout**: 2×3 panel grid
```
         Selectivity          Geometry            MDS
OTC      [bar+points]        [bar+points]       [bar+points]
nonOTC   [bar+points]        [bar+points]       [bar+points]  
Control  [bar+points]        [bar+points]       [bar+points]
```

OR: 3 panels (one per measure), grouped bars by Group, split by Bilateral/Unilateral

**Look-alike**: Ayzenberg et al. 2023, Fig 4C/5C/6C
- Violin plots with bootstrapped distribution of controls
- Individual patient points labeled
- Error bars = SEM
- Significance asterisks

**Style**:
- Colors: Coral/Teal for Bilateral/Unilateral (consistent with your PDF)
- Individual OTC points visible (n=6 per category type)
- p-values displayed on significant comparisons
- Y-axis: measure value; X-axis: Bilateral | Unilateral

**Alternative**: 4-category version (Face, Word, Object, House separately)
- Look-alike: Liu Fig 3D — boxplot with individual patient shapes overlaid
- Shows Object/House elevated in OTC only

---

## Figure 3: RSM Heatmaps — Geometry Visualization

**Purpose**: Directly visualize what "geometry preservation" means

**Layout**: 2×2 grid
```
              T1                    T2
Control   [4×4 heatmap]        [4×4 heatmap]    r = 0.85
OTC       [4×4 heatmap]        [4×4 heatmap]    r = 0.42
```

**Look-alike**: Liu et al. 2025, Fig 3A/B
- 4×4 matrix: Face, Object, House, Word
- Color scale: Pearson correlation (blue-white-red or yellow-blue)
- Diagonal = 1.0 (self-correlation)
- Off-diagonal shows category relationships

**Key elements**:
- Pick representative OTC patient (e.g., one with lowest geometry preservation)
- Pick representative control (average performer)
- Annotation: "Geometry Preservation = correlation between T1 and T2 matrices"
- Show r value for each subject

**Alternative**: Show bilateral ROI (LOC or PPA) vs unilateral ROI (FFA or VWFA)

---

## Figure 4: MDS Embedding — Representational Space

**Purpose**: Visualize MDS shift; show bilateral categories drift more in OTC

**Layout**: 1×2 or 1×3 panels
```
Panel A: Control (representative)     Panel B: OTC (representative)
   
      Word●←─────●Word                    Word●←─────●Word
           Face●←●Face                         Face●←●Face  
    Object●←────────●Object              Object●←────────────────●Object
     House●←────────●House                House●←────────────────●House
     
     [small arrows, balanced]            [large arrows for bilateral]
```

**Look-alike**: Nordt et al. 2023, Fig 3a/d
- 2D scatter with category points
- Small circles = T1, Large circles = T2 (or use arrows)
- Color by category (Face=red, Word=blue, Object=cyan, House=green)
- Procrustes-aligned

**Key elements**:
- Arrow length = MDS shift magnitude
- Show bilateral categories (Object, House) with longer arrows in OTC
- Optional: overlay all 6 OTC patients (different shapes)
- Annotation: "Arrows show T1→T2 movement after Procrustes alignment"

**Quantification panel** (optional):
- Bar graph: Mean arrow length by category type
- Same Bilateral > Unilateral pattern

---

## Figure 5: Spatial Drift Control (Supplementary?)

**Purpose**: Rule out "bilateral categories just moved more" explanation

**Layout**: 
- Panel A: 2D flatmap showing ROI peak locations
- Panel B: Bar graph of spatial drift by category

**Look-alike**: Ayzenberg et al. 2023, Fig 4B/5B
- 2D heatmap of VTC (Anterior-Posterior × Lateral-Medial)
- Darker = more controls activated at that coordinate
- Patient peaks overlaid as labeled points

**Key elements**:
- Show no systematic Bilateral > Unilateral spatial drift
- Word shows most drift (18mm) — but it's unilateral
- Supports: representational degradation ≠ spatial relocation

---

## Tables

### Table 1: Demographics
| | OTC (n=6) | nonOTC (n=9) | Control (n=9) | p |
|---|---|---|---|---|
| Sex (F/M) | 2/4 | 5/4 | 4/5 | |
| Age T1 | 13.5 ± 3.1 | 14.8 ± 2.2 | 11.7 ± 2.4 | .053 |
| Age T2 | 14.7 ± 3.5 | 15.7 ± 2.5 | 13.5 ± 2.2 | |
| Scan interval | 1.2 ± 1.0 | 0.9 ± 0.8 | 1.8 ± 1.0 | .171 |
| Surgery side (L/R) | 4/2 | 4/5 | — | |

### Table 2: Core Results (Q3)
| Measure | Group | Bilateral | Unilateral | Δ | 95% CI | p |
|---|---|---|---|---|---|---|
| Selectivity | OTC | 0.42 | 0.14 | +0.28 | [0.12, 0.43] | <.001* |
| | nonOTC | 0.14 | 0.13 | +0.01 | [-0.07, 0.08] | .793 |
| | Control | 0.26 | 0.15 | +0.11 | [0.04, 0.17] | .002* |
| Geometry | OTC | 0.42 | 0.71 | −0.29 | [-0.51, -0.07] | .009* |
| | nonOTC | 0.73 | 0.76 | −0.04 | [-0.18, 0.11] | .639 |
| | Control | 0.66 | 0.76 | −0.10 | [-0.26, 0.06] | .236 |
| MDS | OTC | 0.38 | 0.30 | +0.08 | [0.01, 0.14] | .016* |
| | nonOTC | 0.25 | 0.20 | +0.04 | [-0.01, 0.10] | .100 |
| | Control | 0.27 | 0.25 | +0.02 | [-0.03, 0.07] | .397 |

### Table 3: Measure Independence (Q6)
| Pair | OTC r | nonOTC r | Control r |
|---|---|---|---|
| Selectivity × Geometry | −0.06 | −0.03 | −0.16 |
| Selectivity × MDS | +0.40 | −0.12 | +0.29 |
| Geometry × MDS | −0.25 | −0.38 | −0.31 |

All |r| < 0.5 → measures are independent

---

## Supplementary Figures

**Fig S1**: Individual patient timelines (Liu Fig 1 style)
**Fig S2**: Category-specific results (Q2) — 4 categories × 3 measures
**Fig S3**: Pairwise comparisons (Q4) — Object vs Face, Object vs Word, etc.
**Fig S4**: Covariate analysis (Q5) — age/scan gap correlations
**Fig S5**: Spatial drift by category (Word=18mm outlier)

---

## Color Scheme (Consistent)

- Bilateral: Coral (#FF6B6B) or Salmon
- Unilateral: Teal (#4ECDC4) or Sky blue
- OTC patients: Warm colors (orange/red shapes)
- nonOTC patients: Cool colors (blue shapes)
- Controls: Gray

- Face: Magenta/Pink
- Word: Orange/Yellow
- Object: Cyan/Blue
- House: Green

(Matches Liu and standard VTC literature conventions)